In [1]:
# imports
from boxoffice.modeling.cleaned_data import X_train, y_train, X_test, y_test
from sklearn.metrics import root_mean_squared_error
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.formula.api as smf

MOVIES_DB_PATH: boxoffice/db/data/data.sqlite
MOVIES_DB_PATH: ../boxoffice/db/data/data.sqlite
movies.csv exists, 1731906929.6094918, 1731906849.873322
Reading from movies.csv
Index([], dtype='object')


In [2]:
X_train['opening_wide_revenue'] = y_train
X_test['opening_wide_revenue'] = y_test
X_train['production_method'] = X_train['production_method'].replace('Stop-Motion Animation', 'Digital Animation')
X_test['production_method'] = X_test['production_method'].replace('Stop-Motion Animation', 'Digital Animation')

/tmp/ipykernel_21636/3335510034.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test['opening_wide_revenue'] = y_test


In [ ]:
formula1 = "opening_wide_revenue ~ 0 + top_5_trailer_views + budget + wikipedia_pre_release_cumulative_views + wikipedia_pre_release_cumulative_views : in_franchise : production_method"
formula2 = "opening_wide_revenue ~ 0 + budget + wikipedia_pre_release_cumulative_views + wikipedia_pre_release_cumulative_views : in_franchise : production_method"
formula3 = "opening_wide_revenue ~ 0 + budget + wikipedia_pre_release_monday_views + wikipedia_pre_release_monday_views : in_franchise : production_method"

formula = formula2

model_simple_glm = smf.glm(formula=formula, data=X_train).fit()

pred_simple_glm = model_simple_glm.predict(X_test)

print("RMSE:", root_mean_squared_error(y_test, pred_simple_glm))
print(model_simple_glm.summary())

train_pred = model_simple_glm.predict(X_train)

training_data = pd.DataFrame(
    {
        "actual": y_train,
        "predicted": train_pred,
        "title": X_train["title"],
        "production_method": X_train["production_method"],
    }
)

# write to a file
training_data.to_csv("training_data.csv")

fig = px.scatter(training_data, x="actual", y="predicted", hover_name="title", color="production_method")
fig.add_trace(go.Scatter(x=training_data["actual"], y=training_data["actual"], mode="lines", name="y=x"))
fig.update_layout(title="Actual vs Predicted Opening Weekend Revenue", xaxis_title="Actual", yaxis_title="Predicted")

RMSE: 21472287.968184467
                  Generalized Linear Model Regression Results                   
Dep. Variable:     opening_wide_revenue   No. Observations:                  677
Model:                              GLM   Df Residuals:                      671
Model Family:                  Gaussian   Df Model:                            5
Link Function:                 Identity   Scale:                      2.9599e+14
Method:                            IRLS   Log-Likelihood:                -12237.
Date:                  Tue, 19 Nov 2024   Deviance:                   1.9861e+17
Time:                          22:11:24   Pearson chi2:                 1.99e+17
No. Iterations:                       3   Pseudo R-squ. (CS):             0.9837
Covariance Type:              nonrobust                                         
                                                                                                   coef    std err          z      P>|z|      [0.025      0.975]
----

In [11]:
results_df = pd.DataFrame({'title': X_test['title'], 'actual': y_test, 'predicted': pred_simple_glm, 'production_method': X_test['production_method']})

# plot actual vs predicted
fig = px.scatter(results_df, x='actual', y='predicted', hover_name='title', color='production_method')
fig.add_trace(go.Scatter(x=results_df['actual'], y=results_df['actual'], mode='lines', name='y=x'))
fig.update_layout(title='Actual vs Predicted Opening Weekend Revenue', xaxis_title='Actual', yaxis_title='Predicted')